# Import 🚀

In [6]:
!pip install -qq torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [7]:
import torch
import random

import networkx as nx
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import plotly.express as px

from itertools import combinations, permutations
from collections import defaultdict
from torch_geometric.data import InMemoryDataset, Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, GATConv, TransformerConv, SAGEConv
from sklearn.metrics import f1_score, balanced_accuracy_score

In [8]:
random.seed(2023)

# Clustering helpers 🐆

In [46]:
# Изменение значений кластеров на 1, кроме кластера-исключения
# ВНИМАНИЕ: напрямую изменяет вес в переданном словаре `values_dict`
def update_rest_values_by_one(values_dict, except_key, add=False):
  increment = 1 if add else -1
  for key, value in values_dict.items():
    if key != except_key:
      if value == 0 and increment > 0 or value > 0:
        values_dict[key] = value + increment

In [47]:
# Поиск кластера с наибольшим числом голосов. Если таких кластеров несколько, вернуть None и список этих кластеров
def find_cluster_assign(values_dict):
  if len(values_dict) == 1:
    # if values_dict[next(iter(values_dict))] == 0:
    #   return None, []
    return max(values_dict, key=values_dict.get), []
  max_value = max(values_dict.values())
  keys_with_max = [key for key, value in values_dict.items() if value == max_value]
  if len(keys_with_max) > 1:
    return None, keys_with_max
  else:
    if max_value == 0:
      return None, keys_with_max
    return keys_with_max[0], []

In [48]:
# Получение кластера для не рассмотренного узла из рассмотренного. Если кластеров с максимум несколько, вернуть их
def get_existing_node_cluster(values_dict):
  node_cluster, keys_with_max = find_cluster_assign(values_dict)
  if node_cluster == None:
    return {key: values_dict[key] for key in keys_with_max}
  else:
    return {node_cluster: 1}

In [49]:
# После посещения ранее не рассмторенного узла, необходимо увеличить вес его кластера
# ВНИМАНИЕ: напрямую изменяет вес в переданном словаре `values_dict`
def update_weight_after_adding_unseen_node(values_dict, node_cluster_dict):
  if len(node_cluster_dict) == 1:
    node_cluster = next(iter(node_cluster_dict))
    values_dict[node_cluster] += 1

In [50]:
# Добавить в кластеры назначение для расмотренной и не рассмотренной вершин
# ВНИМАНИЕ: напрямую изменяет назначение кластеров в переданном словаре `cluster_assign`
def add_nodes_pair_seen_unseen(cluster_assign, first_node, second_node):
  second_node_cluster = get_existing_node_cluster(cluster_assign[first_node])
  cluster_assign[second_node] = second_node_cluster
  update_weight_after_adding_unseen_node(cluster_assign[first_node], second_node_cluster)

In [51]:
# Добавить в кластеры назначение для расмотренной и не рассмотренной вершин не из одного кластера
# ВНИМАНИЕ: напрямую изменяет назначение кластеров в переданном словаре `cluster_assign`
def add_nodes_pair_seen_unseen_not_in(cluster_assign, first_node, second_node, current_cluster):
  node_cluster, _ = find_cluster_assign(cluster_assign[first_node])
  if node_cluster != None:
    cluster_assign[first_node][node_cluster] += 1
  cluster_assign[second_node] = {current_cluster: 1}

In [52]:
# Подтвердить назначение кластеру, увеличив его на 1, а остальные назначения уменьшив на 1
# ВНИМАНИЕ: напрямую изменяет назначение кластеров в переданном словаре `cluster_assign`
def accept_assign_to_cluster(cluster_assign, node, node_cluster):
  cluster_assign[node][node_cluster] += 1
  update_rest_values_by_one(cluster_assign[node], node_cluster)

In [53]:
# Опровергнуть назначение кластеру, уменьшив его на 1, а остальные назначения увеличив на 1
# ВНИМАНИЕ: напрямую изменяет назначение кластеров в переданном словаре `cluster_assign`
def discard_assign_to_cluster(cluster_assign, node, node_cluster):
  if cluster_assign[node][node_cluster] > 0:
    cluster_assign[node][node_cluster] -= 1
  update_rest_values_by_one(cluster_assign[node], node_cluster, add=True)

In [54]:
# Добавить новый кластер или увеличить его значение, а остальные назначения уменьшить на 1
# Если передана еще одна вершина, для нее значение этого кластера уменьшить на 1
# ВНИМАНИЕ: напрямую изменяет назначение кластеров в переданном словаре `cluster_assign`
def add_new_cluster(cluster_assign, node, new_cluster, another_node=None):
  if another_node != None and cluster_assign[another_node][new_cluster] > 0:
    cluster_assign[another_node][new_cluster] -= 1
  if new_cluster in cluster_assign[node]:
    cluster_assign[node][new_cluster] += 1
  else:
    cluster_assign[node][new_cluster] = 1
  update_rest_values_by_one(cluster_assign[node], new_cluster)

In [55]:
# Уменьшает значение кластера для одной вершины, а другой увеличивает
# ВНИМАНИЕ: напрямую изменяет назначение кластеров в переданном словаре `cluster_assign`
def downvote_cluster(cluster_assign, node, new_cluster, another_node=None):
  cluster_assign[another_node][new_cluster] += 1
  if new_cluster in cluster_assign[node] and cluster_assign[node][new_cluster] > 0:
    cluster_assign[node][new_cluster] -= 1
  update_rest_values_by_one(cluster_assign[node], new_cluster, add=True)

In [56]:
# Увеличивает значение кластера той вершине, где он максимален.
# Для другой убираем остальные назначения и оставляем только этот кластер.
# ВНИМАНИЕ: напрямую изменяет назначение кластеров в переданном словаре `cluster_assign`
def assign_to_max_cluster(cluster_assign, first_node, second_node, max_cluster):
  cluster_assign[first_node][max_cluster] += 1
  if max_cluster in cluster_assign[second_node]:
    cluster_assign[second_node][max_cluster] += 1
  else:
    cluster_assign[second_node][max_cluster] = 1
  cluster_value = cluster_assign[second_node][max_cluster]
  cluster_assign[second_node].clear()
  cluster_assign[second_node][max_cluster] = cluster_value

In [57]:
def perform_clustering(G, pairs_classification, with_permutations):
  cluster_assign = {}
  current_cluster = 0

  for index, pair in enumerate(permutations(G.nodes(), 2) if with_permutations else combinations(G.nodes(), 2)):
    first_node = pair[0]
    second_node = pair[1]

    # Обе вершины классифицированы как принадлежащие одному кластеру
    if pairs_classification[index] == 1:
      # Обе вершины уже были встречены ранее
      if first_node in cluster_assign and second_node in cluster_assign:
        # Получим их текущие назначения кластеров
        first_node_cluster, _ = find_cluster_assign(cluster_assign[first_node])
        second_node_cluster, _ = find_cluster_assign(cluster_assign[second_node])
        # Если назначения кластеров равны и они оба не None, то мы подтверждаем назначение этому кластеру,
        # то есть увеличиваем его на 1, а остальные назначения уменьшаем на 1
        if first_node_cluster == second_node_cluster and first_node_cluster != None:
          accept_assign_to_cluster(cluster_assign, first_node, first_node_cluster)
          accept_assign_to_cluster(cluster_assign, second_node, second_node_cluster)
        # Назначения кластерам не равны
        if first_node_cluster != second_node_cluster:
          # Оба кластера известны, тогда обеим вершинам увеличиваем кластер другой вершины, остальные уменьшаем
          # ИЛИ
          # assign_to_max_cluster: Отнести вершину к кластеру с максимумом
          if first_node_cluster != None and second_node_cluster != None:
            # add_new_cluster(cluster_assign, first_node, second_node_cluster)
            # add_new_cluster(cluster_assign, second_node, first_node_cluster)
            if cluster_assign[first_node][first_node_cluster] > cluster_assign[second_node][second_node_cluster]:
              assign_to_max_cluster(cluster_assign, first_node, second_node, first_node_cluster)
            elif cluster_assign[first_node][first_node_cluster] < cluster_assign[second_node][second_node_cluster]:
              assign_to_max_cluster(cluster_assign, second_node, first_node, second_node_cluster)
          # Один кластер известен, а другой нет. Тогда для вершины с известным кластером вес уменьшаем, 
          # так как эта ситуация не дает нам уверенности в принадлежности к этому кластеру. 
          # Для вершины с неизвестным кластером увеличиваем (или добавляем) вес известному кластеру, остальные уменьшаем
          if first_node_cluster != None and second_node_cluster == None:
            add_new_cluster(cluster_assign, second_node, first_node_cluster, first_node)
          if first_node_cluster == None and second_node_cluster != None:
            add_new_cluster(cluster_assign, first_node, second_node_cluster, second_node)
      # Обеим вершинам не были назначены кластеры
      if first_node not in cluster_assign and second_node not in cluster_assign:
        cluster_assign[first_node] = {current_cluster: 1}
        cluster_assign[second_node] = {current_cluster: 1}
        current_cluster += 1
      # Первой вершине был назначен кластер, а второй еще нет
      if first_node in cluster_assign and second_node not in cluster_assign:
        add_nodes_pair_seen_unseen(cluster_assign, first_node, second_node)
      # Второй вершине был назначен кластер, а первой еще нет
      if second_node in cluster_assign and first_node not in cluster_assign:
        add_nodes_pair_seen_unseen(cluster_assign, second_node, first_node)

    # Вершины не были классифицированы как принадлежащие одному кластеру
    if pairs_classification[index] == 0:
      # Обе вершины уже были встречены ранее
      if first_node in cluster_assign and second_node in cluster_assign:
        # Получим их текущие назначения кластеров
        first_node_cluster, _ = find_cluster_assign(cluster_assign[first_node])
        second_node_cluster, _ = find_cluster_assign(cluster_assign[second_node])
        # Если назначения кластеров равны и они оба не None, то мы уменьшаем назначение этому кластеру, 
        # а остальные назначения увеличиваем на 1. При этом второй вершине назначаем следующий
        # после наибольшего для первой вершины кластер
        if first_node_cluster == second_node_cluster and first_node_cluster != None:
          discard_assign_to_cluster(cluster_assign, first_node, first_node_cluster)
          discard_assign_to_cluster(cluster_assign, second_node, second_node_cluster)
          if first_node_cluster + 1 not in cluster_assign[second_node]:
            cluster_assign[second_node][first_node_cluster + 1] = 1
          else:
            cluster_assign[second_node][first_node_cluster + 1] += 1
        # Назначения кластерам не равны
        if first_node_cluster != second_node_cluster:
          # Оба кластера известны, тогда обеим вершинам увеличиваем свой кластер, остальные уменьшаем
          # if first_node_cluster != None and second_node_cluster != None:
          #   accept_assign_to_cluster(cluster_assign, first_node, first_node_cluster)
          #   accept_assign_to_cluster(cluster_assign, second_node, second_node_cluster)
          # Один кластер известен, а другой нет. Тогда для вершины с известным кластером вес увеличиваем.
          # Для вершины с неизвестным кластером уменьшаем вес известному кластеру, остальные увеличиваем
          if first_node_cluster != None and second_node_cluster == None:
            downvote_cluster(cluster_assign, second_node, first_node_cluster, first_node)
          if first_node_cluster == None and second_node_cluster != None:
            downvote_cluster(cluster_assign, first_node, second_node_cluster, second_node)
      # Обеим вершинам не были назначены кластеры
      if first_node not in cluster_assign and second_node not in cluster_assign:
        cluster_assign[first_node] = {current_cluster: 1}
        cluster_assign[second_node] = {current_cluster + 1: 1}
        current_cluster += 2
      # Первой вершине был назначен кластер, а второй еще нет, 
      # назначить вершине следующий кластер после макимального для первой
      if first_node in cluster_assign and second_node not in cluster_assign:
        # add_nodes_pair_seen_unseen_not_in(cluster_assign, 
        #                                   first_node, 
        #                                   second_node, 
        #                                   current_cluster)
        # current_cluster += 1
        first_node_cluster, _ = find_cluster_assign(cluster_assign[first_node])
        if first_node_cluster != None:
          cluster_assign[second_node] = {first_node_cluster + 1: 1}
        else:
          cluster_assign[second_node] = {current_cluster: 1}
          current_cluster += 1
      # Второй вершине был назначен кластер, а первой еще нет
      if second_node in cluster_assign and first_node not in cluster_assign:
        # add_nodes_pair_seen_unseen_not_in(cluster_assign, 
        #                                   second_node, 
        #                                   first_node, 
        #                                   current_cluster)
        # current_cluster += 1
        second_node_cluster, _ = find_cluster_assign(cluster_assign[second_node])
        if second_node_cluster != None:
          cluster_assign[first_node] = {second_node_cluster + 1: 1}
        else:
          cluster_assign[first_node] = {current_cluster: 1}
          current_cluster += 1

    # print(f'Pair: {pair}. Classification: {pairs_classification[index]}. Assign: {first_node} {cluster_assign[first_node]} {second_node} {cluster_assign[second_node]}')
      


  return cluster_assign

In [58]:
def get_nodes_clusters(cluster_assign):
  nodes_clusters = {}
  for node, assign in cluster_assign.items():
    nodes_clusters[node] = max(assign, key=assign.get)

  return nodes_clusters

In [59]:
def restore_missing_clusters(cluster_assign):
  clusters = list(cluster_assign.values())
  full_sequence = set(range(0, clusters[-1] + 1))
  clusters_set = set(clusters)
  missing_clusters = list(clusters_set ^ full_sequence)

  for key, value in cluster_assign.items():
    for missing in missing_clusters:
      if value > missing:
        cluster_assign[key] -= 1
      else:
        break

In [60]:
def clustering_pipeline(G, pairs_classification, with_permutations):
  cluster_assign = perform_clustering(G, pairs_classification, with_permutations)
  nodes_clusters = get_nodes_clusters(cluster_assign)
  restore_missing_clusters(nodes_clusters)
  return nodes_clusters

In [61]:
# cluster_assign = {
#     1: 0,
#     2: 0,
#     3: 2,
#     4: 2,
#     5: 4,
#     6: 4,
#     7: 6,
#     8: 7
# }

In [62]:
# node_pairs = [(1, 2), (1, 3), (1, 4), (1, 5),
#               (2, 1), (2, 3), (2, 4), (2, 5),
#               (3, 1), (3, 2), (3, 4), (3, 5),
#               (4, 1), (4, 2), (4, 3), (4, 5),
#               (5, 1), (5, 2), (5, 3), (5, 4)]
# pairs_classification = [1, 0, 0, 0,
#                         0, 0, 0, 0,
#                         0, 0, 1, 1,
#                         0, 0, 1, 0,
#                         0, 0, 0, 1]

# Prepare graphs data ⛲

In [63]:
def get_graph_true_clusters(G):
  clusters_dict = {}

  for i in range(len(G.graph["partition"])):
    for node in G.graph["partition"][i]:
      clusters_dict[node] = i

  return clusters_dict

In [64]:
def get_graph_node_pairs(G, with_permutations):
  degrees_dict = dict(G.degree())

  initial_embeddings = []

  nodes_combinations = permutations(G.nodes(), 2) if with_permutations else combinations(G.nodes(), 2)

  for nodes_pair in nodes_combinations:
    first_node = nodes_pair[0]
    second_node = nodes_pair[1]

    degrees_pair = [degrees_dict[first_node], degrees_dict[second_node]]

    initial_embeddings.append(degrees_pair)

  embeddings = np.array(initial_embeddings)

  return embeddings

In [65]:
def get_graphs_embeddings(graphs, with_permutations):
  embeddings_all = []

  for graph in graphs:
    embeddings = get_graph_node_pairs(graph, with_permutations)
    embeddings_all.append(embeddings)

  return embeddings_all

# Dataset creation 🍚

In [66]:
BATCH_SIZE = 8

In [67]:
class GraphDataset(InMemoryDataset):
    def __init__(self, graphs, embeddings):
        super(GraphDataset, self).__init__('.', None, None, None)

        data_graphs = []

        for index, graph in enumerate(graphs):
          adj = nx.to_scipy_sparse_array(graph).tocoo()
          row = torch.from_numpy(adj.row.astype(np.int64)).to(torch.long)
          col = torch.from_numpy(adj.col.astype(np.int64)).to(torch.long)
          edge_index = torch.stack([row, col], dim=0)

          x = torch.from_numpy(embeddings[index]).type(torch.float32)

          data = Data(edge_index=edge_index,
                      num_nodes=graph.number_of_nodes(),
                      x=x,
                      num_classes=2)
          
          data_graphs.append(data)

        self.data, self.slices = self.collate([data_graphs])

    def _download(self):
        return

    def _process(self):
        return

    def __repr__(self):
        return '{}()'.format(self.__class__.__name__)

# Load model 🧴

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [27]:
# GCN model with 2 layers 
class ConvNet(torch.nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = GCNConv(2, 64)
        self.conv2 = GCNConv(64, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [28]:
class GATNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(2, 64)
        self.conv2 = GATConv(64, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [29]:
class TransformerNet(torch.nn.Module):
  def __init__(self):
        super().__init__()
        self.conv1 = TransformerConv(2, 256, dropout=0.6) 
        self.conv2 = TransformerConv(256, 2, dropout=0.5) 

  def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [30]:
class SageNet(torch.nn.Module):
  def __init__(self):
        super().__init__()
        self.conv1 = SAGEConv(2, 64) 
        self.conv2 = SAGEConv(64, 2)

  def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [31]:
def evaluate(model, loader):
  model.eval()

  with torch.no_grad():
    for loader_ in loader:
      for data in loader_:
        data = data.to(device)
        output = model(data)
        _, preds = torch.max(output, dim=1)
    
    return preds

In [32]:
conv = ConvNet()
conv.load_state_dict(torch.load('/content/drive/MyDrive/data/conv_model'))

<All keys matched successfully>

In [33]:
gat = GATNet()
gat.load_state_dict(torch.load('/content/drive/MyDrive/data/gat_model'))

<All keys matched successfully>

In [34]:
transformer = TransformerNet()
transformer.load_state_dict(torch.load('/content/drive/MyDrive/data/transformer_model'))

<All keys matched successfully>

In [35]:
sage = SageNet()
sage.load_state_dict(torch.load('/content/drive/MyDrive/data/sage_model'))

<All keys matched successfully>

In [36]:
models_dict = {
    'GCNConv': conv,
    'GATConv': gat,
    'TransformerConv': transformer,
    'SAGEConv': sage
}

In [37]:
def get_graph_classification(model, size, probs):
  graph = nx.stochastic_block_model(size, probs, seed=0)

  embeddings = get_graphs_embeddings([graph])

  dataset = GraphDataset([graph], embeddings)

  dataset_loader = DataLoader(dataset, batch_size=BATCH_SIZE)

  preds = evaluate(model, dataset_loader)

  return graph, preds

In [68]:
def get_graph_data_loader(size, probs, with_permutations):
  graph = nx.stochastic_block_model(size, probs, seed=0)

  embeddings = get_graphs_embeddings([graph], with_permutations)

  dataset = GraphDataset([graph], embeddings)

  dataset_loader = DataLoader(dataset, batch_size=BATCH_SIZE)

  return graph, dataset_loader

# Perform clustering 📖

In [41]:
CLUSTERS_PROBS_TEST_3 = [[0.75, 0.015, 0.0002], [0.015, 0.85, 0.0075], [0.0002, 0.0075, 0.90]]

In [42]:
CLUSTERS_SIZES_TEST_3 = [[20, 50, 30],
                         [150, 40, 110],
                         [300, 150, 50],
                         [150, 450, 90],
                         [450, 300, 200],
                         [500, 400, 300]]

In [75]:
CLUSTERS_PROBS_TEST_5 = [[0.75, 0.05, 0.02, 0.01, 0.05], 
             [0.05, 0.85, 0.07, 0.08, 0.01], 
             [0.02, 0.07, 0.90, 0.03, 0.04],
             [0.01, 0.08, 0.03, 0.65, 0.10],
             [0.05, 0.01, 0.04, 0.10, 0.95]]

In [76]:
CLUSTERS_SIZES_TEST_5 = [
                    [10, 100, 15, 40, 15],
                    [100, 70, 10, 30, 20],
                    [200, 100, 150, 50, 90],
                    [220, 50, 260, 120, 70],
                    [200, 250, 100, 300, 90],
                    [200, 450, 250, 250, 350]
                    ]

In [84]:
CLUSTERS_PROBS_TEST_10 = [[0.75, 0.05, 0.02, 0.01, 0.05, 0.007, 0.02, 0.3, 0.25, 0.015], 
            [0.05, 0.85, 0.07, 0.08, 0.01, 0.001, 0.03, 0.0, 0.04, 0.02], 
            [0.02, 0.07, 0.90, 0.03, 0.04, 0.03, 0.09, 0.017, 0.003, 0.01],
            [0.01, 0.08, 0.03, 0.65, 0.10, 0.005, 0.10, 0.025, 0.05, 0.1],
            [0.05, 0.01, 0.04, 0.10, 0.95, 0.085, 0.07, 0.06, 0.077, 0.009],
            [0.007, 0.001, 0.03, 0.005, 0.085, 0.70, 0.2, 0.0001, 0.08, 0.0015],
            [0.02, 0.03, 0.09, 0.10, 0.07, 0.2, 0.5, 0.3, 0.19, 0.0002],
            [0.3, 0.0, 0.017, 0.025, 0.06, 0.0001, 0.3, 0.99, 0.001, 0.15],
            [0.25, 0.04, 0.003, 0.05, 0.077, 0.08, 0.19, 0.001, 0.87, 0.035],
            [0.015, 0.02, 0.01, 0.1, 0.009, 0.0015, 0.0002, 0.15, 0.035, 0.85]]

In [82]:
CLUSTERS_SIZES_TEST_10 = [
                    [30, 10, 20, 15, 5, 10, 40, 30, 50, 10],
                    [10, 100, 15, 40, 15, 90, 60, 50, 110, 20],
                    [100, 70, 10, 30, 20, 150, 90, 80, 170, 50],
                    [220, 50, 260, 120, 70, 40, 105, 15, 25, 75],
                    [200, 100, 100, 100, 110, 90, 80, 120, 150, 50],
                    [300, 100, 200, 150, 50, 250, 150, 100, 50, 50]
                    ]

In [69]:
def evaluate_graphs_nodes_clustering(models, sizes, probs, with_permutations=False):
  metrics = {
    'GCNConv': [],
    'GATConv': [],
    'TransformerConv': [],
    'SAGEConv': []
}

  graphs_test = []

  for size in sizes:
    graph, dataset_loader = get_graph_data_loader(size, probs, with_permutations)
    graphs_test.append(graph)
    for model_name, model in models.items():     
      nodes_classification = evaluate(model, dataset_loader)
      pred_clusters = list(clustering_pipeline(graph, nodes_classification.numpy(), with_permutations).values())
      true_clusters = list(get_graph_true_clusters(graph).values())

      f1 = f1_score(true_clusters, pred_clusters, average='weighted')

      metrics[model_name].append(f1)

    # print(f'Graph nodes: {len(graph.nodes())}; F1-Score:{f1} Accuracy: {accuracy}')

  graphs_nodes_amount = [len(graph.nodes()) for graph in graphs_test]

  metrics_df = pd.DataFrame(metrics, index=graphs_nodes_amount)

  return metrics_df, graphs_test

## Clusters = 3

In [70]:
metrics_df_3, graphs_3 = evaluate_graphs_nodes_clustering(models_dict, 
                                                          CLUSTERS_SIZES_TEST_3, 
                                                          CLUSTERS_PROBS_TEST_3,
                                                          True)

In [71]:
metrics_df_3

,GCNConv,GATConv,TransformerConv,SAGEConv
100,0.956698,0.942866,0.938826,0.952658
300,0.410080,0.420837,0.420837,0.462676
500,1.000000,1.000000,1.000000,0.003987
690,0.559006,0.005721,1.000000,1.000000
950,1.000000,0.355263,0.998947,0.393250
1200,0.297619,0.297619,0.297619,0.456949


In [72]:
fig = px.line(metrics_df_3, title="Метрики моделей для графов с 3 кластерами").update_layout(xaxis_title="Количество узлов",
                                                                                             yaxis_title="Значение метрики")   
fig.show()

## Clusters = 5

In [77]:
metrics_df_5, graphs_5 = evaluate_graphs_nodes_clustering(models_dict, CLUSTERS_SIZES_TEST_5, CLUSTERS_PROBS_TEST_5)

In [78]:
metrics_df_5

,GCNConv,GATConv,TransformerConv,SAGEConv
180,0.014242,0.013226,0.776064,0.473098
230,0.184679,0.315372,0.196726,0.186640
590,0.341104,0.248331,0.246533,0.090369
720,0.009218,0.677072,0.689522,0.132840
940,0.256419,0.450873,0.451616,0.006800
1500,0.002369,0.113210,0.460444,0.113655


In [79]:
fig = px.line(metrics_df_5, title="Метрики моделей для графов с 5 кластерами").update_layout(xaxis_title="Количество узлов",
                                                                                             yaxis_title="Значение метрики")   
fig.show()

## Clusters = 10

In [85]:
metrics_df_10, graphs_10 = evaluate_graphs_nodes_clustering(models_dict, CLUSTERS_SIZES_TEST_10, CLUSTERS_PROBS_TEST_10)

In [87]:
metrics_df_10

,GCNConv,GATConv,TransformerConv,SAGEConv
220,0.062862,0.051715,0.017269,0.043729
510,0.004257,0.030070,0.014656,0.015759
770,0.009363,0.055976,0.009403,0.005137
980,0.009753,0.213920,0.406458,0.007847
1100,0.036884,0.025422,0.149253,0.039846
1400,0.005674,0.015106,0.382053,0.315261


In [86]:
fig = px.line(metrics_df_10, title="Метрики моделей для графов с 10 кластерами").update_layout(xaxis_title="Количество узлов",
                                                                                             yaxis_title="Значение метрики")   
fig.show()

In [ ]:
for graph in graphs_10:
  print(f'Nodes: {len(graph.nodes())} Edges: {len(graph.edges())}')

Nodes: 50 Edges: 157
Nodes: 120 Edges: 984
Nodes: 220 Edges: 4567
Nodes: 465 Edges: 16407
Nodes: 510 Edges: 22289
Nodes: 635 Edges: 33754
Nodes: 770 Edges: 53727
Nodes: 900 Edges: 64117
Nodes: 980 Edges: 81075
Nodes: 1330 Edges: 134353
Nodes: 1350 Edges: 129323
Nodes: 1100 Edges: 93147
Nodes: 1400 Edges: 148366


# CD Lib comparison 🍃

In [88]:
!pip install -qq cdlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.6/228.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 34.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 55.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 59.7 MB/s eta 0:00:00


In [89]:
from cdlib import algorithms

Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'karateclub', 'graph_tool', 'infomap', 'leidenalg'}
Note: to be able to use all overlapping methods, you need to install some additional packages:  {'ASLPAw', 'karateclub'}
Note: to be able to use all bipartite methods, you need to install some additional packages:  {'infomap', 'leidenalg', 'wurlitzer'}


In [133]:
cd_lib_models = {
    'louvain': algorithms.louvain,
    'greedy_modularity': algorithms.greedy_modularity,
    'walktrap': algorithms.walktrap
}

In [91]:
def get_cd_lib_pred_clusters(com_partition):
  clusters_dict = {}

  for i in range(len(com_partition)):
    for node in com_partition[i]:
      clusters_dict[node] = i

  return clusters_dict

In [134]:
def perform_cd_lib_clustering(models, sizes, probs):
  metrics = {
    'louvain': [],
    'greedy_modularity': [],
    'walktrap': []
  }

  graphs_test = []

  for size in sizes:
    graph = nx.stochastic_block_model(size, probs, seed=0)
    graphs_test.append(graph)
    for model_name, model in models.items():     
      communities = model(graph)
      pred_clusters = list(get_cd_lib_pred_clusters(communities.communities).values())
      # print(len(pred_clusters))
      true_clusters = list(get_graph_true_clusters(graph).values())

      f1 = f1_score(true_clusters, pred_clusters, average='weighted')

      metrics[model_name].append(f1)

    # print(f'Graph nodes: {len(graph.nodes())}; F1-Score:{f1} Accuracy: {accuracy}')

  graphs_nodes_amount = [len(graph.nodes()) for graph in graphs_test]

  metrics_df = pd.DataFrame(metrics, index=graphs_nodes_amount)

  return metrics_df

## Clusters = 3

In [135]:
cd_lib_metrics_df_3 = perform_cd_lib_clustering(cd_lib_models, CLUSTERS_SIZES_TEST_3, CLUSTERS_PROBS_TEST_3)

In [136]:
cd_lib_metrics_df_3

,louvain,greedy_modularity,walktrap
100,0.604286,0.604286,0.604286
300,0.766667,0.766667,0.766667
500,1.000000,1.000000,1.000000
690,0.565217,0.565217,0.565217
950,1.000000,1.000000,1.000000
1200,1.000000,1.000000,1.000000


In [137]:
fig = px.line(cd_lib_metrics_df_3, title="Метрики CD Lib моделей для графов с 3 кластерами").update_layout(xaxis_title="Количество узлов",
                                                                                                           yaxis_title="Значение метрики")   
fig.show()

## Clusters = 5

In [138]:
cd_lib_metrics_df_5 = perform_cd_lib_clustering(cd_lib_models, CLUSTERS_SIZES_TEST_5, CLUSTERS_PROBS_TEST_5)

In [139]:
cd_lib_metrics_df_5

,louvain,greedy_modularity,walktrap
180,0.081786,0.009259,0.236941
230,0.753623,0.718841,0.753623
590,0.661212,0.661212,0.661212
720,0.365484,0.330021,0.330021
940,0.585106,0.460052,0.585106
1500,0.535082,0.535082,0.535082


In [141]:
fig = px.line(cd_lib_metrics_df_5, title="Метрики CD Lib моделей для графов с 5 кластерами").update_layout(xaxis_title="Количество узлов",
                                                                                                           yaxis_title="Значение метрики")   
fig.show()

## Clusters = 10


In [143]:
cd_lib_metrics_df_10 = perform_cd_lib_clustering(cd_lib_models, CLUSTERS_SIZES_TEST_10, CLUSTERS_PROBS_TEST_10)

In [144]:
cd_lib_metrics_df_10

,louvain,greedy_modularity,walktrap
220,0.074380,0.058862,0.096257
510,0.003040,0.001783,0.003268
770,0.076394,0.076620,0.092764
980,0.219581,0.147426,0.220245
1100,0.132231,0.132231,0.139860
1400,0.183673,0.160514,0.171429


In [145]:
fig = px.line(cd_lib_metrics_df_10, title="Метрики CD Lib моделей для графов с 10 кластерами").update_layout(xaxis_title="Количество узлов",
                                                                                                           yaxis_title="Значение метрики")   
fig.show()